# Customer Purchase Behavior Clustering
- This notebook performs clustering on customer purchase behavior data to identify distinct purchase patterns.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import seaborn as sns
# pca
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score



In [ ]:

purchase_behavior_features = [
    'NumDealsPurchases',  # Promo purchases
    'NumWebPurchases',  # Online
    'NumCatalogPurchases',  # Catalog
    'NumStorePurchases',  # In-store
    'NumWebVisitsMonth'  # Website visits (optional: inverse indicator)
]

df = pd.read_csv('featured_customer_segmentation.csv')
# Display the first few rows of the DataFrame
print("DataFrame Head:")
print(df.head())

In [ ]:
scaler = StandardScaler()
X_behavior = scaler.fit_transform(df[purchase_behavior_features])

inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_behavior)
    inertia.append(kmeans.inertia_)

print(inertia)

plt.plot(range(1, 10), inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for purchase Pattern Clustering')
plt.grid(True)
plt.show()


In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df['PurchaseCluster'] = kmeans.fit_predict(X_behavior)

# Print cluster distributions 
cluster_counts = df['PurchaseCluster'].value_counts().sort_index()
print("\nNumber of customers in each cluster:")
print(cluster_counts)
print("\nPercentage of customers in each cluster:")
print((cluster_counts / len(df) * 100).round(2), "%")

# Plot distribution
plt.figure(figsize=(10, 6))
cluster_counts.plot(kind='bar')
plt.title('Distribution of Customers Across Clusters')
plt.xlabel('Cluster')
plt.ylabel('Number of Customers')
plt.xticks(rotation=0)
for i, v in enumerate(cluster_counts):
    plt.text(i, v, str(v), ha='center', va='bottom')
plt.tight_layout()
plt.show()


In [ ]:

# Organize features by category
feature_categories = {
    'Demographics': [ 'Education', 'Total_Dependents',
                    'Teenhome', 'Kidhome'],
    'Expenses': [ 'MntWines', 'MntFruits', 'MntMeatProducts',
                'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'],
    'Purchase Channels': ['NumWebPurchases', 'NumStorePurchases', 'NumCatalogPurchases',
                         'NumDealsPurchases','NumWebVisitsMonth'],
    'cash': ['Total_Spending','Income'],
    'channel_ratios': ['Web_Ratio', 'Store_Ratio', 'Catalog_Ratio', 'Deals_Ratio'],
    'Expense_ratios': ['Wine_Ratio', 'Fruit_Ratio', 'Meat_Ratio',
                         'Fish_Ratio', 'Sweet_Ratio', 'Gold_Ratio']
}

# Color palettes for each category
color_palettes = {
     'cash': 'Purples'  ,
    'Demographics': 'Blues',

    'Purchase Channels': 'Oranges',

    'channel_ratios': 'Reds',
    'Expenses': 'Greens',
    'Expense_ratios': 'Greys'

}


# Plot each category
for category_name, features in feature_categories.items():
    # Calculate means by cluster
    cluster_means = df.groupby('PurchaseCluster')[features].mean().round(2)
    print(f"\n{category_name.upper()} MEANS BY CLUSTER:")
    print(cluster_means)

    # Create figure with appropriate size based on feature count
    fig_width = min(14, max(10, len(features)))
    fig, ax = plt.subplots(figsize=(fig_width, 8))

    # Plot the transposed data for better visualization
    cluster_means.T.plot(kind='bar', ax=ax, colormap=color_palettes[category_name])

    # Add styling
    plt.title(f'{category_name} by Purchase Cluster', fontsize=16, pad=20)
    plt.ylabel('Average Value', fontsize=12)
    plt.xlabel('Feature', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Cluster', fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.3)

    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f', padding=3, fontsize=8)

    plt.tight_layout()
    plt.show()



In [ ]:
# Create radar chart comparing key metrics across clusters
# Select representative metrics from each category
key_metrics = ['Age', 'Income', 'Education', 'Total_Spending',
              'NumWebPurchases' ,'NumWebVisitsMonth','NumStorePurchases', 'NumCatalogPurchases',]

# Get data and normalize for radar chart
radar_data = df.groupby('PurchaseCluster')[key_metrics].mean()

# Normalize each metric to 0-100 scale
for col in radar_data.columns:
    radar_data[col] = 100 * radar_data[col] / radar_data[col].max()

# Set up radar chart
angles = np.linspace(0, 2*np.pi, len(key_metrics), endpoint=False).tolist()
angles += angles[:1]  # Close the circle

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

# Plot each cluster
for cluster in radar_data.index:
    values = radar_data.loc[cluster].tolist()
    values += values[:1]  # Close the polygon

    ax.plot(angles, values, linewidth=2, label=f'Cluster {cluster}')
    ax.fill(angles, values, alpha=0.1)

# Add labels
labels = [x.replace('_', ' ') for x in key_metrics]
labels += [labels[0]]  # Complete the circle of labels
ax.set_xticks(angles)
ax.set_xticklabels(labels)

# Customize chart
ax.set_title('Key Metrics by Purchase Cluster (Normalized)', size=16, pad=20)
ax.grid(True)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN


# Apply DBSCAN on the standardized purchase behavior data
# Adjust eps and min_samples based on your data
dbscan = DBSCAN(eps=1.0, min_samples=5)

# Fit DBSCAN model
df['DBSCAN_Cluster'] = dbscan.fit_predict(X_behavior)

# Count clusters (-1 indicates noise points)
cluster_counts = df['DBSCAN_Cluster'].value_counts().sort_index()
print("DBSCAN Cluster Counts:")
print(cluster_counts)

# Calculate percentage of data points in each cluster
total_points = len(df)
cluster_percentages = (cluster_counts / total_points * 100).round(2)
print("\nDBSCAN Cluster Percentages:")
for cluster, percentage in cluster_percentages.items():
    status = "Noise points" if cluster == -1 else f"Cluster {cluster}"
    print(f"{status}: {percentage}%")

# Apply PCA for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_behavior)

# Visualize the DBSCAN clusters with PCA
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['DBSCAN_Cluster'],
                      cmap='viridis', s=60, alpha=0.7, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='DBSCAN Cluster')
plt.title('DBSCAN Clustering of Purchase Behavior (PCA View)', fontsize=16, pad=20)
plt.xlabel(f'First Principal Component ({pca.explained_variance_ratio_[0]:.1%} variance)')
plt.ylabel(f'Second Principal Component ({pca.explained_variance_ratio_[1]:.1%} variance)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Compare DBSCAN clusters with KMeans clusters
cross_tab = pd.crosstab(df['DBSCAN_Cluster'], df['PurchaseCluster'],
                        normalize='index') * 100
print("\nCross-tabulation (%) of DBSCAN vs. KMeans clusters:")
print(cross_tab.round(2))

# Compare purchase patterns across DBSCAN clusters
purchase_metrics = purchase_behavior_features + ['Total_Purchases', 'Income', 'Total_Spending']

# Get cluster summaries, handling -1 (noise) separately
cluster_summary = df.groupby('DBSCAN_Cluster')[purchase_metrics].mean().round(2)
print("\nPurchase Behavior by DBSCAN Cluster:")
print(cluster_summary)

# Visualize key metrics by DBSCAN cluster
metrics_to_plot = ['NumWebPurchases', 'NumStorePurchases', 'NumCatalogPurchases', 'NumDealsPurchases']

plt.figure(figsize=(14, 8))
ax = cluster_summary[metrics_to_plot].plot(kind='bar', figsize=(14, 8))
plt.title('Purchase Channel Usage by DBSCAN Cluster', fontsize=16, pad=20)
plt.xlabel('DBSCAN Cluster (-1 = Noise)')
plt.ylabel('Average Number of Purchases')
plt.legend(title='Purchase Channel')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f', padding=3, fontsize=9)

plt.tight_layout()
plt.show()

# Characterize the clusters based on their features
print("\nDBSCAN Cluster Characteristics:")
for cluster in sorted(df['DBSCAN_Cluster'].unique()):
    if cluster == -1:
        print("\nNoise Points:")
    else:
        print(f"\nCluster {cluster}:")

    # Get top distinctive features
    cluster_data = cluster_summary.loc[cluster]

    # For non-noise clusters, find distinguishing features
    if cluster != -1:
        other_clusters = cluster_summary.drop(cluster)


In [ ]:


# Get average ratios per cluster
channel_ratios = df.groupby('PurchaseCluster')[['Web_Ratio', 'Store_Ratio', 'Catalog_Ratio', 'Deals_Ratio']].mean()

# Create bar plot
ax = channel_ratios.plot(kind='bar', figsize=(10, 6), width=0.8)
plt.title('Preferred Purchase Channels by Cluster')
plt.xlabel('Cluster')
plt.ylabel('Proportion of Purchases')
plt.legend(title='Channel')
plt.xticks(rotation=0)

# Add value labels on bars
for i in ax.containers:
    ax.bar_label(i, fmt='%.2f', padding=3)

plt.tight_layout()
plt.show()

# Print favorite channel for each cluster
for cluster in range(len(channel_ratios)):
    favorite_channel = channel_ratios.iloc[cluster].idxmax()
    ratio = channel_ratios.iloc[cluster][favorite_channel]
    print(f"Cluster {cluster} favorite channel: {favorite_channel.replace('_Ratio', '')} ({ratio:.2%})")


In [ ]:

# 1. Compute your correlations
features = [
    'Income', 'Age', 'Total_Dependents', 'Tenure_Days',
    'Teenhome', 'Kidhome', 'Education',
    'Marital_Together', 'Marital_Single', 'Marital_Divorced', 'Marital_Widow',
    'PurchaseCluster'
]
corr = df[features].corr()

# 2. Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# 3. Plot
plt.figure(figsize=(14, 12))
sns.heatmap(
    corr,
    mask=mask,
    annot=True,
    fmt='.2f',
    linewidths=.5,
    square=True,
    cbar_kws={'shrink': .8, 'label': 'Pearson ρ'},
    vmin=-1, vmax=1,
    center=0,
    cmap='vlag'   # a red‑to‑blue diverging map; you could switch back to 'coolwarm' if you prefer
)

# 4. Styling
plt.title('Feature Correlation Matrix (Lower Triangle)', fontsize=18, pad=20)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)

# 5. Interpretation legend
plt.gcf().text(
    0.01, 0.01,
    "■ Strong positive (> 0.7)\n■ Strong negative (< -0.7)\n■ Near zero: weak/no linear relation",
    fontsize=10,
    bbox=dict(facecolor='white', alpha=0.8)
)

plt.tight_layout()
plt.show()

print(corr)


In [ ]:




# -- Compute derived feature --
df['weighted_total_dependents'] = df['Kidhome'] * 2 + df['Teenhome']

# -- Feature variants --
feature_sets = {
    'v1': ['Income', 'Age', 'Education', 'Total_Dependents'],
    'v2': ['Income', 'Age', 'Education', 'Teenhome', 'Kidhome'],
    'v3': ['Income', 'Age', 'Education',
           'Marital_Together', 'Marital_Single', 'Marital_Divorced', 'Marital_Widow',
           'Total_Dependents', 'Teenhome', 'Kidhome'],
    'v4': ['Income', 'Age', 'Education', 'Kidhome'],
    'v5': ['Income', 'Age', 'Education', 'weighted_total_dependents']
}

y = df['PurchaseCluster']

# -- Hyperparameter grid for RF --
param_grid_rf = {
    'n_estimators': [100, 300],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# -- Cross-validation setup --
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# -- Storage for summary results --
summary = []

# -- Explore both Logistic Regression and RF Classifier for each variant --
for variant, feats in feature_sets.items():
    print(f"\n=== Variant: {variant} | Features: {feats} ===")
    X = df[feats]

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    # -- Logistic Regression --
    lr = LogisticRegression(max_iter=1000)
    # CV Accuracy
    lr_cv_acc = cross_val_score(lr, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)
    print(f"LogisticRegression CV Accuracy: {lr_cv_acc.mean() * 100:.2f}% ± {lr_cv_acc.std() * 100:.2f}%")
    # Test-set Accuracy
    lr.fit(X_train_s, y_train)
    lr_preds = lr.predict(X_test_s)
    lr_acc = accuracy_score(y_test, lr_preds) * 100
    print(f"LogisticRegression Test Accuracy: {lr_acc:.2f}%")

    # -- Random Forest Classifier --
    rf = RandomForestClassifier(random_state=42)
    rf_cv_acc = cross_val_score(rf, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)
    print(f"RandomForest CV Accuracy: {rf_cv_acc.mean() * 100:.2f}% ± {rf_cv_acc.std() * 100:.2f}%")

    grid = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train_s, y_train)
    best_rf = grid.best_estimator_
    rf_preds = best_rf.predict(X_test_s)
    rf_acc = accuracy_score(y_test, rf_preds) * 100
    print(f"RandomForest Test Accuracy: {rf_acc:.2f}%")
    print(f"Best RF Params: {grid.best_params_}")

    # Save summary
    summary.append({
        'Variant': variant,
        'LR_CV_Acc': lr_cv_acc.mean() * 100,
        'LR_Test_Acc': lr_acc,
        'RF_CV_Acc': rf_cv_acc.mean() * 100,
        'RF_Test_Acc': rf_acc
    })

# -- Summary DataFrame --
summary_df = pd.DataFrame(summary)
print("\n=== Summary of All Variants (Percentage Accuracy) ===")
print(summary_df)


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# -- Log-transform skewed Income --
df['Income'] = np.log1p(df['Income'])

# -- Weighted dependents feature --
df['weighted_total_dependents'] = df['Kidhome'] * 2 + df['Teenhome']

# -- Feature sets (Education is ordinal) --
feature_sets = {
    'v1': ['Income', 'Age', 'Education', 'Total_Dependents'],
    'v2': ['Income', 'Age', 'Education', 'Teenhome', 'Kidhome'],
    'v3': ['Income', 'Age', 'Education',
           'Marital_Together', 'Marital_Single', 'Marital_Divorced', 'Marital_Widow',
           'Total_Dependents', 'Teenhome', 'Kidhome'],
    'v4': ['Income', 'Age', 'Education', 'Kidhome'],
    'v5': ['Income', 'Age', 'Education', 'weighted_total_dependents']
}

y = df['PurchaseCluster']

# -- Hyperparameter grids --
param_grid_rf = {
    'n_estimators': [100, 300],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

# -- Stratified Cross-validation --
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

summary = []

for variant, feats in feature_sets.items():
    print(f"\n=== Variant: {variant} | Features: {feats} ===")
    X = df[feats]

    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    # Scaling
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    # -- Logistic Regression (tuned) --
    grid_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=kf, scoring='accuracy', n_jobs=-1)
    grid_lr.fit(X_train_s, y_train)
    best_lr = grid_lr.best_estimator_
    lr_preds = best_lr.predict(X_test_s)
    lr_acc = accuracy_score(y_test, lr_preds) * 100
    lr_cv_acc = cross_val_score(best_lr, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)
    print(f"LogisticRegression CV Accuracy: {lr_cv_acc.mean()*100:.2f}% ± {lr_cv_acc.std()*100:.2f}%")
    print(f"LogisticRegression Test Accuracy: {lr_acc:.2f}%")
    print("Best LR Params:", grid_lr.best_params_)

    # Confusion matrix for LR
    print("confusion_matrix for lr:")

    print(confusion_matrix(y_test, lr_preds))

    # -- Random Forest Classifier (tuned) --
    grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=kf, scoring='accuracy', n_jobs=-1)
    grid_rf.fit(X_train_s, y_train)
    best_rf = grid_rf.best_estimator_
    rf_preds = best_rf.predict(X_test_s)
    rf_acc = accuracy_score(y_test, rf_preds) * 100
    rf_cv_acc = cross_val_score(best_rf, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)
    print(f"RandomForest CV Accuracy: {rf_cv_acc.mean()*100:.2f}% ± {rf_cv_acc.std()*100:.2f}%")
    print(f"RandomForest Test Accuracy: {rf_acc:.2f}%")
    print("Best RF Params:", grid_rf.best_params_)


    print("confusion_matrix for rf:")
    print(confusion_matrix(y_test, rf_preds))

    # -- Store results --
    summary.append({
        'Variant': variant,
        'LR_CV_Acc': lr_cv_acc.mean() * 100,
        'LR_Test_Acc': lr_acc,
        'RF_CV_Acc': rf_cv_acc.mean() * 100,
        'RF_Test_Acc': rf_acc
    })

# -- Summary Results --
summary_df = pd.DataFrame(summary)
print("\n=== Summary of All Variants (Final) ===")
print(summary_df)


In [ ]:

# -- Log-transform skewed Income --
df['Income'] = np.log1p(df['Income'])

# -- Weighted dependents feature --
df['weighted_total_dependents'] = df['Kidhome'] * 2 + df['Teenhome']

# -- v3 Feature set --
features_v3 = [
    'Income', 'Age', 'Education',
    'Marital_Together', 'Marital_Single', 'Marital_Divorced', 'Marital_Widow',
    'Total_Dependents', 'Teenhome', 'Kidhome'
]
X = df[features_v3]
y = df['PurchaseCluster']

# -- Hyperparameter grid for Random Forest --
param_grid_rf = {
    'n_estimators': [100, 300],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# -- Stratified Cross-validation --
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# -- Train/Test Split --
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# -- Scaling --
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# -- Random Forest Classifier (tuned) --
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=kf, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train_s, y_train)
best_rf = grid_rf.best_estimator_
rf_preds = best_rf.predict(X_test_s)
rf_acc = accuracy_score(y_test, rf_preds) * 100
rf_cv_acc = cross_val_score(best_rf, scaler.fit_transform(X), y, scoring='accuracy', cv=kf)

print(f"RandomForest CV Accuracy: {rf_cv_acc.mean()*100:.2f}% ± {rf_cv_acc.std()*100:.2f}%")
print(f"RandomForest Test Accuracy: {rf_acc:.2f}%")
print("Best RF Params:", grid_rf.best_params_)

print("confusion_matrix for rf:")
print(confusion_matrix(y_test, rf_preds))

# -- Summary Results --
summary = [{
    'Variant': 'v3',
    'RF_CV_Acc': rf_cv_acc.mean() * 100,
    'RF_Test_Acc': rf_acc
}]
summary_df = pd.DataFrame(summary)
print("\n=== Summary for v3 Random Forest ===")
print(summary_df)

